In [1]:
import time
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.linalg import svd

In [2]:
topk = 10
rank_list = [i+1 for i in range(topk)]
latent = 50

In [3]:
%%time
ML100K_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
dataset = pd.read_csv(
  ML100K_URL, 
  names=["user_id", "item_id", "rating", "timestamp"], 
  sep="\t"
)
matrix_data = csr_matrix(
  (dataset.rating, 
  (dataset.user_id, dataset.item_id))
)

CPU times: user 52.8 ms, sys: 41.1 ms, total: 93.9 ms
Wall time: 570 ms


In [4]:
%%time
# SVD分解を行う
u, s, v = svd(
  matrix_data.toarray(), 
  full_matrices=False
)

CPU times: user 2.03 s, sys: 387 ms, total: 2.41 s
Wall time: 1.46 s


In [5]:
print(u.shape)
print(s.shape)
print(v.shape)

(944, 944)
(944,)
(944, 1683)


In [6]:
sk = np.array(s)
# 次元削減
sk[latent:] = 0
# 評価行列を復元する
usv = np.dot(
  np.dot(u, np.diag(sk)), v
)

In [7]:
usv[1:5, 1:5].round(3)

array([[ 6.448e+00,  2.950e+00,  1.626e+00,  3.045e+00],
       [ 2.334e+00,  1.160e-01, -1.010e-01,  3.100e-01],
       [ 3.280e-01, -2.740e-01, -1.440e-01, -1.570e-01],
       [ 3.830e-01, -4.350e-01,  4.400e-02,  6.000e-03]])

In [8]:
# すでに評価したユーザ-アイテムの組み合わせを取り除く
recommend_matrix = np.where(
  matrix_data.toarray(), 0, usv
)

In [9]:
%%time
# ユニークなユーザーIDのリストを作成する
uq_users = np.sort(dataset.user_id.unique().tolist())

# 推薦結果を格納するテーブルを作成する
df_recommend_list = pd.DataFrame(
  columns=['user_id', 'item_id', 'score', 'rank']
)

# 各ユーザーに対して、トップ10アイテムを絞り込む
for user_id in uq_users:

  # 対象ユーザーの未接触アイテムへのスコア情報を取得
  item_scores = recommend_matrix[user_id]

  df_recommend = pd.DataFrame()
  df_recommend['user_id'] = [user_id] * topk
  # トップ10アイテムのアイテムIDを取得する
  df_recommend['item_id'] = \
    np.argsort(item_scores)[::-1][:topk]
  # トップ10アイテムのスコアを取得する
  df_recommend['score'] = \
    np.sort(item_scores)[::-1][:topk]
  df_recommend["rank"] = rank_list

  # 全体テーブルに格納する
  df_recommend_list = \
    df_recommend_list.append(
        df_recommend, 
        ignore_index=True
    )

CPU times: user 4.92 s, sys: 68.8 ms, total: 4.98 s
Wall time: 6.01 s


In [10]:
df_recommend_list

,user_id,item_id,score,rank
0,1,423,3.480956,1
1,1,403,3.107652,2
2,1,732,2.901051,3
3,1,357,2.767992,4
4,1,385,2.608116,5
...,...,...,...,...
9425,943,735,2.562035,6
9426,943,684,2.447764,7
9427,943,742,2.282232,8
9428,943,357,2.246284,9
